In [1]:
import torch
from tqdm import tqdm
import atomizer
from hydra import initialize_config_dir, compose
import os
import mdct
from omegaconf import OmegaConf
%env CUDA_LAUNCH_BLOCKING=1

# loading the config file
abs_config_dir=os.path.abspath("../scripts/conf")
with initialize_config_dir(version_base=None, config_dir=abs_config_dir):
    cfg=compose(config_name="config.yaml")


env: CUDA_LAUNCH_BLOCKING=1


In [2]:
# build a dataloader (looks like .batch does not do several workers ?)
datapipe, loader, synth = atomizer.data.get_data(cfg)

In [4]:
from pytorch_lightning.utilities.model_summary import summarize

device =  'cuda' if torch.cuda.is_available() and cfg.training.cuda else 'cpu'

# sets the right number of times
cfg.atoms.num_classes.times=int(next(iter(datapipe))[0]['num_times'])

# build the model
model = atomizer.AtomsLocatorMDCT.load_from_checkpoint('/home/antoine/repositories/atomizer/notebooks/atomizer/3u6rfnu3/checkpoints/last.ckpt')

# summarize
print(summarize(model, max_depth=1))


/home/antoine/anaconda3/envs/1.11/lib/python3.9/site-packages/torch/utils/data/datapipes/iter/combining.py:180: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  warnings.warn("Some child DataPipes are not exhausted when __iter__ is called. We are resetting "


TypeError: __init__() missing 3 required positional arguments: 'num_classes', 'arch_cfg', and 'training_cfg'

In [4]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
import wandb

# create a logger, a lr monitor and a checkpoint callback
wandb_logger = WandbLogger(project="atomizer")
lr_monitor = LearningRateMonitor(logging_interval='step')
model_checkpoint = ModelCheckpoint(save_last=True, every_n_train_steps=1000)

wandb_logger.experiment.config.update(OmegaConf.to_container(cfg))

trainer = Trainer(
    amp_backend="native",
    precision=32,
    accelerator='gpu',
    devices=1,
    detect_anomaly=True,
    logger=wandb_logger,
    accumulate_grad_batches=2,
    callbacks=[lr_monitor, model_checkpoint]
)
trainer.fit(model, loader)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: liutkus. Use `wandb login --relogin` to force relogin


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/antoine/anaconda3/envs/1.11/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:122: UserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn("You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type              | Params
-----------------------------------------------------
0 | abs_encoding   | ModuleDict        | 785 K 
1 | tf_spe         | PointSineEmbedder | 540   
2 | masks          | ParameterDict     | 1.8 K 
3 | mlp_pe         | Mlp               | 259 K 
4 | predictors     | ModuleDict        | 5.7 M 
5 | encoder_blocks | ModuleList        | 7.8 M 
6 | norm           | LayerNorm         | 720   
7 | decoder_blocks | ModuleList        | 10.4 M
-----------------------------------------------

Training: 0it [00:00, ?it/s]

/home/antoine/anaconda3/envs/1.11/lib/python3.9/site-packages/torch/utils/data/datapipes/iter/combining.py:180: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  warnings.warn("Some child DataPipes are not exhausted when __iter__ is called. We are resetting "
/home/antoine/anaconda3/envs/1.11/lib/python3.9/site-packages/torch/utils/data/datapipes/iter/combining.py:180: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  warnings.warn("Some child DataPipes are not exhausted when __iter__ is called. We are resetting "
/home/antoine/anaconda3/envs/1.11/lib/python3.9/site-packages/torch/utils/data/datapipes/iter/combining.py:180: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read fro

RuntimeError: CUDA error: device-side assert triggered

In [ ]:
from tqdm import tqdm

for b in tqdm(zip(loader, range(100))):
        pass

In [ ]:
import matplotlib.pyplot as plt
import IPython